In [ ]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from wrf import vinterp, getvar, destagger
from numba import njit

nx = 2000
ny = 1800
nz = 90
ti = 4
G = 9.81
zbot=0.2
ztop=18
z = np.linspace(zbot,ztop,nz)


In [ ]:
nc_ctl = Dataset('/p/work1/lloveras/adj_4km/long_runs/ctl_48h/wrfout_d01_2021-01-05_00_00_00')
nc_adj = Dataset('/p/work1/lloveras/adj_4km/long_runs/adj_48h/wrfout_d01_2021-01-05_00_00_00')
nc_wave = Dataset('/p/work1/lloveras/adj_4km/long_runs/wave_48h/wrfout_d01_2021-01-05_00_00_00')

pv_ctl = np.asarray(getvar(nc_ctl,'pvo',timeidx=ti))*1e-6
pv_adj = np.asarray(getvar(nc_adj,'pvo',timeidx=ti))*1e-6
pv_wave = np.asarray(getvar(nc_wave,'pvo',timeidx=ti))*1e-6
u_ctl = np.asarray(getvar(nc_ctl,'ua',timeidx=ti))
u_adj = np.asarray(getvar(nc_adj,'ua',timeidx=ti))
u_wave = np.asarray(getvar(nc_wave,'ua',timeidx=ti))
v_ctl = np.asarray(getvar(nc_ctl,'va',timeidx=ti))
v_adj = np.asarray(getvar(nc_adj,'va',timeidx=ti))
v_wave = np.asarray(getvar(nc_wave,'va',timeidx=ti))
p = np.asarray(getvar(nc_ctl,'p',units='Pa',timeidx=ti))
th = np.asarray(getvar(nc_ctl,'th',timeidx=ti))

pv_ctl = vinterp(nc_ctl,pv_ctl,'ght_msl',z,timeidx=ti)
pv_adj = vinterp(nc_adj,pv_adj,'ght_msl',z,timeidx=ti)
pv_wave = vinterp(nc_wave,pv_wave,'ght_msl',z,timeidx=ti)
u_ctl = vinterp(nc_ctl,u_ctl,'ght_msl',z,timeidx=ti)
u_adj = vinterp(nc_adj,u_adj,'ght_msl',z,timeidx=ti)
u_wave = vinterp(nc_wave,u_wave,'ght_msl',z,timeidx=ti)
v_ctl = vinterp(nc_ctl,v_ctl,'ght_msl',z,timeidx=ti)
v_adj = vinterp(nc_adj,v_adj,'ght_msl',z,timeidx=ti)
v_wave = vinterp(nc_wave,v_wave,'ght_msl',z,timeidx=ti)
p = vinterp(nc_ctl,p,'ght_msl',z,timeidx=ti)
th = vinterp(nc_ctl,th,'ght_msl',z,timeidx=ti)


In [ ]:
dp_dt = np.zeros((nz,ny,nx))
for k in range(1,nz-1):
    dp_dt[k,:,:] = (p[k+1,:,:] - p[k-1,:,:])/(th[k+1,:,:] - th[k-1,:,:])
    
dp_dt[0,:,:] = (p[1,:,:] - p[0,:,:])/(th[1,:,:] - th[0,:,:])
dp_dt[-1,:,:] = (p[-1,:,:] - p[-2,:,:])/(th[-1,:,:] - th[-2,:,:])

sig = (-1/G)*dp_dt
dqgpv_adj = sig*(pv_adj - pv_ctl)
dqgpv_wave = sig*(pv_wave - pv_ctl)
du_adj = u_adj - u_ctl
dv_wave = v_wave - v_ctl
du_wave = u_wave - u_ctl
dv_adj = v_adj - v_ctl


In [ ]:
np.save('/p/work1/lloveras/adj_4km/processed_data/qgpv/dqgpv_adj',dqgpv_adj)
np.save('/p/work1/lloveras/adj_4km/processed_data/qgpv/dqgpv_wave',dqgpv_wave)
np.save('/p/work1/lloveras/adj_4km/processed_data/qgpv/du_adj',du_adj)
np.save('/p/work1/lloveras/adj_4km/processed_data/qgpv/dv_adj',du_adj)
np.save('/p/work1/lloveras/adj_4km/processed_data/qgpv/du_wave',du_wave)
np.save('/p/work1/lloveras/adj_4km/processed_data/qgpv/dv_wave',dv_wave)
